In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import importlib
from scipy.ndimage import median_filter
import sys
import os
import struct
import pywt
import concurrent.futures
import glob
from windrose import WindroseAxes
import cmcrameri as cmc
from cmcrameri import cm
from scipy.stats import linregress
from scipy.signal import welch
from datetime import datetime


sys.path.append(os.path.join(os.getcwd(), 'EC'))
sys.path.append(os.path.join(os.getcwd(), 'plotting'))
sys.path.append(os.path.join(os.getcwd(), 'MO'))
sys.path.append(os.path.join(os.getcwd(), 'SPC'))

import constants
import Func_read_data
import Func_despike_data
import sensor_info
import Func_DR
import Func_MRFD
import Funcs_plots
import normalize
import Func_MO

importlib.reload(Func_read_data)
importlib.reload(Func_despike_data)
importlib.reload(constants)
importlib.reload(sensor_info)
importlib.reload(Func_DR)
importlib.reload(Func_MRFD)
importlib.reload(Funcs_plots)
importlib.reload(normalize)
importlib.reload(Func_MO)

from Func_read_data import *
from constants import *
from Func_despike_data import *
from sensor_info import *
from Func_MRFD import *
from Func_DR import *
from Funcs_plots import *
from normalize import *
from Func_MO import *

In [18]:

def load_fastdata(folder, start, end):
    start_dt = datetime.strptime(start, '%Y-%m-%d_%H:%M:%S')
    end_dt = datetime.strptime(end, '%Y-%m-%d_%H:%M:%S')
    all_dfs = []
    for root, dirs, files in os.walk(folder):
        files = sorted(files)
        for file in files:
            if file.endswith('.dat'):
                try:
                    date_str = file.split('_')[-2] + '_' + file.split('_')[-1].split('.')[0]
                    file_dt = datetime.strptime(date_str, '%Y-%m-%d_%H%M')
                    if start_dt <= file_dt <= end_dt:
                        file_path = os.path.join(root, file)
                        print(f"Processing file: {file_path}")
                        df = pd.read_csv(file_path, index_col=0, parse_dates=True, sep='\t')
                        df.index = pd.to_datetime(df.index, format='%Y-%m-%d %H:%M:%S.%f', errors='coerce')
                        all_dfs.append(df)
                except Exception as e:
                    print(f"Skipping {file}: {e}")
    if all_dfs:
        return pd.concat(all_dfs)
    else:
        return pd.DataFrame()


In [24]:
start='2024-05-01_00:00:00'
end='2024-05-04_00:00:00'
path='/home/engbers/Documents/PhD/EC_data_convert/SFC_DR/'
fastdata_SFC=load_fastdata(path, start, end)
path='/home/engbers/Documents/PhD/EC_data_convert/CSAT_16m_DR/'
fastdata_16=load_fastdata(path, start, end)
path='/home/engbers/Documents/PhD/EC_data_convert/CSAT_26m_DR/'
fastdata_26=load_fastdata(path, start, end)

Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0000.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0100.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0200.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0300.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0400.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0500.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0600.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_2024-05-01_0700.dat
Processing file: /home/engbers/Documents/PhD/EC_data_convert/SFC_DR/202405/SFC_Fastdata_proc_202

In [25]:
fastdata_16

,Ux,Uy,Uz,Ts
timestamp,,,,
2024-05-01 00:00:00.000,0.41293,0.64846,0.02167,-18.572770
2024-05-01 00:00:00.100,0.45529,0.61531,-0.02410,-18.601430
2024-05-01 00:00:00.200,0.45623,0.62937,-0.04358,-18.552080
2024-05-01 00:00:00.300,0.50287,0.63643,-0.02827,-18.575970
2024-05-01 00:00:00.400,0.48913,0.62979,-0.01526,-18.591860
...,...,...,...,...
2024-05-04 00:59:59.500,4.99262,0.98039,-0.08374,-9.549561
2024-05-04 00:59:59.600,5.03702,1.03761,-0.10210,-9.581940
2024-05-04 00:59:59.700,5.00574,0.92185,-0.26963,-9.706635
